# Test Notebook for Curved Lines

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import random as rand

import sys

sys.path.append('../')
from a_art_functions import *

In [ ]:
def paint_line_curved(canvas, color):
    num_points = np.random.randint(3,6)
    
    x = np.random.randint(0,canvas.shape[0],size=num_points)
    while x.shape[0] > np.unique(x).shape[0]:
        x = np.unique(x)
        x = np.concatenate([x, np.random.randint(0,canvas.shape[0],size=1)])
    x.sort()
    y = np.random.randint(0,canvas.shape[1],size=num_points)

    x2 = np.linspace(x[0], x[-1], (x[-1] - x[0])*4)
    x2.sort()
    x2, indices = np.unique(x2, return_index=True)
    y2 = interpolate.pchip_interpolate(x, y, x2[indices])

    
    #do splatter
    for i in range(len(x2)):
        if y2[i] < canvas.shape[1]:
            canvas[int(math.ceil(x2[i]))][int(math.ceil(y2[i]))] = color
        canvas[int(math.floor(x2[i]))][int(math.floor(y2[i]))] = color
        
        
def paint_splat(canvas, color):
    num_points = np.random.randint(3,6)
    splat_width = 4
    splat_height = 20
    
    x = np.random.randint(0,canvas.shape[0],size=num_points)
    while (min(abs(np.ediff1d(x))) < 20) | (max(x) - min(x) <= ((splat_width + 1) * num_points)) | (x.shape[0] != np.unique(x).shape[0]) | (x[1] - x[0] <= splat_width) | (x[-1] - x[-2] <= splat_width):
        x = np.random.randint(0,canvas.shape[0],size=num_points)
    x.sort()
    y = np.random.randint(0,canvas.shape[1],size=num_points)
    
    v, w = x.copy(), y.copy()
    
    for i in range(num_points - 2):
        v_temp = v[i + 1] + random.randint(-splat_width, splat_width)
        while v_temp in v:
            v_temp = v[i + 1] + random.randint(-splat_width, splat_width)
        v[i + 1,] = min(max(v_temp,0),canvas.shape[0]-1)
        w[i + 1,] += min(max(random.randint(-splat_height, splat_height),0),canvas.shape[1]-1)
    v.sort()
        

    line_res = 5
    x2 = np.linspace(x[0], x[-1], (x[-1] - x[0])*line_res)
    x2, indices = np.unique(x2, return_index=True)
    y2 = interpolate.pchip_interpolate(x, y, x2[indices])
    
    v2 = np.linspace(v[0], v[-1], (v[-1] - v[0])*line_res)
    v2, indices = np.unique(v2, return_index=True)
    w2 = interpolate.pchip_interpolate(v, w, v2[indices])
    
    
    #do splatter
    for i in range(len(x2)):
        if y2[i] < canvas.shape[1]:
            canvas[int(math.ceil(x2[i]))][int(math.ceil(y2[i]))] = color
        canvas[int(math.floor(x2[i]))][int(math.floor(y2[i]))] = color
        
        if (v2[i] >= 0) & (w2[i] >= 0) & (v2[i] < canvas.shape[0]) & (w2[i] < canvas.shape[1] - 1):
            canvas[int(math.ceil(v2[i]))][int(math.ceil(w2[i]))] = color
            canvas[int(math.floor(v2[i]))][int(math.floor(w2[i]))] = color
        
        
        
def cy_twombly(width, height, num_colors, num_splatters):
    canvas = np.zeros((width,height),dtype=int)
    print(canvas.shape)

    for x in range(0,num_splatters):
        #randint is inclusive, need to subtract one from num_colors
        color = random.randint(0,num_colors-1)
        paint_splat(canvas, color)

    return canvas

In [ ]:
palette = []
palette.append((206, 179, 146))
palette.append((218, 189, 157))
palette.append((225, 223, 212))
palette.append((19, 20, 15))
palette.append((11, 7, 22))
palette.append((36, 74, 114))
palette.append((193, 175, 84))
palette.append((222, 90, 65))

In [ ]:
width = 800 # 8 inches
height = int(width * (93.5/155))
num_splatters = 20
painting = jackson_pollack_curved(width, height, len(palette), num_splatters)

In [ ]:
picture = create_pollack_image(canvas=painting,palette=palette)
picture

In [ ]:
picture.save('/Users/walkerrdavis/Desktop/scribbles_01.png','PNG')

In [ ]:
x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')

In [ ]:
def get_line(x1, y1, x2, y2):
    points = []
    issteep = abs(y2-y1) > abs(x2-x1)
    if issteep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    rev = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        rev = True
    deltax = x2 - x1
    deltay = abs(y2-y1)
    error = int(deltax / 2)
    y = y1
    ystep = None
    if y1 < y2:
        ystep = 1
    else:
        ystep = -1
    for x in range(x1, x2 + 1):
        if issteep:
            points.append((y, x))
        else:
            points.append((x, y))
        error -= deltay
        if error < 0:
            y += ystep
            error += deltax
    # Reverse the list if the coordinates were reversed
    if rev:
        points.reverse()
    return points

In [ ]:
def w_get_points(p1, p2, p3):
#     print(p1, p2, p3)
    x0, x1 = abs(p2[0] - p1[0]), abs(p3[0] - p2[0])
    
    ly0 = np.linspace(p1[1], p2[1], x0)
    ly1 = np.linspace(p2[1], p3[1], x1)
    l = np.concatenate([ly0,ly1])
    
    return l

In [ ]:
l = [0,rand.randint(0,100)]
c = [rand.randint(0,100),rand.randint(0,100)]
r = [100, rand.randint(0,100)]

x = np.arange(0,100)
y = w_get_points(l, c, r)
f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')

xnew = np.linspace(0, 99, num=400, endpoint=True)

plt.plot(x,y)
# plt.plot(xnew, f(xnew))
plt.plot(xnew, f2(xnew))

plt.xlim(0,100)
plt.ylim(0,100)
plt.show()

In [ ]:
# l = [0,rand.uniform(0,1)]
# c = [rand.uniform(0,1),rand.uniform(0,1)]
# r = [1.0, rand.uniform(0,1)]

# x = [0, rand.uniform(0,1), 1.0]
# y = np.random.uniform(size=3)
print(x)
print(y)

f = interp1d(x, y)
f2 = interp1d(x, y, kind='cubic')
xnew = np.linspace(0, 1, num=7)
print(xnew)

plt.plot(x, y, 'g')
plt.plot(xnew, f(xnew), 'o')
# plt.plot(xnew,f2(xnew))
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

In [ ]:
x = np.arange(0,100)
p = np.zeros([100],dtype=int)
for i in range(3):
    p[rand.randint(0,100)] = rand.randint(0,100)

f = interp1d(x, p)
plt.plot(x, p)
plt.plot(x,f(p))
# plt.xlim(0,100)
# plt.ylim(0,100)
# plt.show()

In [ ]:
w = np.random.uniform(0, 1.0, size=3)
h = np.random.uniform(0, 1.0, size=3)
f = interp1d(w, h)

# f2 = interp1d(w, h, kind='cubic')
# xnew = np.linspace(0, 10, num=3, endpoint=True)

plt.plot(w, h)
plt.plot(f)
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()


In [ ]:
w = np.random.randint(101, size=3)


plt.plot(p0, p1)
plt.xlim(0,100)
plt.ylim(0,100)
plt.show()

In [ ]:
p0 = [rand.randint(0,100),rand.randint(0,100)]
p1 = [rand.randint(0,100),rand.randint(0,100)]

plt.plot(p0, p1)
plt.xlim(0,100)
plt.ylim(0,100)
plt.show()